## Hands-on session 2.1 - working with kinetic simulations: Epperlein-Short test

This session covers basic electron kinetics features in ReMKiT1D. It is not meant to guide the reader through detailed model construction, and most of the model is prebuilt. 

A companion file `es_models.py` is supplied that creates the necessary kinetic models/terms using `common_models`. 

The reader is required to complete the setup of the grid and the variables.

Demonstrated concepts:

- Setting up a full ReMKiT1D grid (x,h,v)
- Parallelization in harmonics
- Distribution variables
- Moments and related derivations
- More prebuilt derivations
- Inspecting kinetic data 

In [1]:
from RMK_support import RKWrapper ,Grid, Node, treeDerivation
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds

from es_models import addESModels,kappa0
import numpy as np
import holoviews as hv 
import panel as pn
import matplotlib.pyplot as plt


### Wrapper initialization

In [2]:
rk = RKWrapper()

### Global parameters for writing the files

In [3]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/day_2_1/"
rk.setHDF5Path(hdf5Filepath) 

### MPI

In kinetic simulations, we have the additional option to add processors in the harmonic direction. If you use 4 or more harmonics (`lMax>2`) try increasing numProcsH (keep in mind that the total number of harmonics should be divisible by numProcsH!)

The total number of MPI processes ReMKiT1D will expect then will be `numProcsX*numProcsH`.

In [4]:
rk.setMPIData(numProcsX=4,numProcsH=1) # use numProcsH to change the number of harmonic direction processes

### Normalization

Let's set a higher normalization temperature than usual

In [5]:
rk.setNormDensity(1.0e19) #n_0
rk.setNormTemperature(100.0) #T_0
rk.setNormRefZ(1.0) # reference ion charge for e-i collision time

### Grid initialization

We now initialize the full grid, where we need to specify the spatial and velocity grids, as well as the number of distribution function harmonics

Complete the initialization of the grid. Make sure it is periodic.

In [6]:
Grid?

Init signature:
Grid(
    xGrid: numpy.ndarray,
    vGrid: numpy.ndarray = array([1.]),
    lMax=0,
    mMax=0,
    interpretXGridAsWidths=False,
    interpretVGridAsWidths=False,
    isPeriodic=False,
    isLengthInMeters=False,
)
Docstring:      Class containing x and v-grid data
Init docstring:
Grid constructor

Args:
    xGrid (numpy.ndarray): x coordinates of each spatial grid cell or their widths
    vGrid (numpy.ndarray): v coordinates of each velocity grid cell or their widths. Defaults to a single cell (effectively no v-grid)
    lMax (int): Maximum l harmonic number. Defaults to 0.
    mMax (int, optional): Maximum m harmonic number. Defaults to 0.
    interpretXGridAsWidths (bool, optional): If True interprets xGrid as cell widths. Defaults to False.
    interpretVGridAsWidths (bool, optional): If True interprets vGrid as cell widths. Defaults to False.
File:           /usr/local/lib/python3.8/dist-packages/RMK_support/grid.py
Type:           type
Subclasses:     

In [7]:
dx = 150.0 # Cell width in reference e-i collision mfps (use this to control the perturbation wavelength in the ES test - larger numbers are more collisional) 

xGrid = dx*np.ones(64)

# Velocity grid setup 
vGrid = [0.0307]
for i in range(1,80):
    vGrid.append(vGrid[i-1]*1.025)
vGrid = np.array(vGrid)

lMax = 1 # Highest used l harmonic (the total number of harmonics is lMax+1, including l=0)

gridObj = Grid(
    xGrid, vGrid, 
    interpretXGridAsWidths=True,
    interpretVGridAsWidths=True,
    lMax = lMax, isPeriodic=True
    ) # YOUR CODE HERE [Ignore mMax - it is unused in ReMKiT1D currently]

L = sum(xGrid) # Length of the spatial grid

rk.grid = gridObj

### Variables

We will need the following variables for the Epperlein-Short test:

- The electron distribution function variable
- The density and the temperature of our electrons 
- An electric field variable

Let's start by setting the initial values of our variables. 

**Note**: Here we use the fact that the distribution function is normalized to $n_0/v_{th}^3$ by default, and that $eT_0=m_ev_{th}^2/2$.

In [8]:
n = np.ones(gridObj.numX())
T = 1.0 + 0.001*np.sin(2*np.pi*gridObj.xGrid/L) # A small temperature perturbation around the reference value T_0=100eV

# A Maxwellian with the above n and T for the l=0 harmonic and 0 for all the others
f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = np.pi**(-1.5) * T[i] ** (-1.5) * n[i] * np.exp(-gridObj.vGrid**2/T[i])
    
# f is [cell index, harmonic index, distribution]

In [9]:
f.shape

(64, 2, 80)

##### Some numerical considerations

We have calculated the discretized values of our analytical Maxwellian on the velocity grid we supplied. Let's check whether the numerical integration in ReMKiT1D will have the correct density. We can do this using the `velocityMoment` function of the `Grid` class.

In [10]:
gridObj.velocityMoment?

Signature:
gridObj.velocityMoment(
    distFun: numpy.ndarray,
    momentOrder: int,
    momentHarmonic=1,
) -> numpy.ndarray
Docstring:
Return velocity moment of distribution function or single harmonic variable

Args:
    distFun (np.ndarray): Distribution or single harmonic variable values
    momentOrder (int): Moment order
    momentHarmonic (int, optional): Harmonic index (Fortran 1 indexing) to take moment of in case of distribution variable. Defaults to 1.

Returns:
    np.ndarray: Moment represented as a contracted array
File:      /usr/local/lib/python3.8/dist-packages/RMK_support/grid.py
Type:      method

In [11]:
# I guess here we pick the order and the harmonic index (starting from 1 because of Fortran indexing)
numerical_dens = gridObj.velocityMoment(distFun=f,momentOrder=0,momentHarmonic=1) # Note, we use the Fortran indexing here (the first harmonic is the l=0 harmonic)
numerical_dens[0]

0.9999491861884808

As you can see, and might expect, this is not equal to the density we requested. Some discretization errors can be corrected easily by rescaling the distribution function:

In [12]:
# Fortran indexes from 1, which is why there has to be a -1 near getH

for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = n[i] *f[i,gridObj.getH(0)-1,:]/numerical_dens[i]

Check that the 0-th moment of the first (l=0) harmonic of f now has the correct value

In [13]:
gridObj.velocityMoment(distFun=f,momentOrder=0,momentHarmonic=1) # Note, we use the Fortran indexing here (the first harmonic is the l=0 harmonic)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

We can now proceed to add the distribution function to the wrapper. Add the distribution function "f" and its dual variable.

In [14]:
# Make sure to indicate f is a distribution.
rk.addVarAndDual("f", f, isDistribution = True, isCommunicated = True)

**NOTE**: Distribution variables on staggered grids are treated the following way:

- The even harmonics of "f" live on cell centers, and the odd live on cell edges
- The opposite is true for "f_dual" - even harmonics are interpolated onto cell edges, odd harmonics are interpolated into cell centers  

We also need to use a number of [built-in derivations](https://remkit1d-python.readthedocs.io/en/latest/custom_fluid.html#Textbook-objects-and-derivations). In particular we want to be able do derive the electron temperature. For this we pass the electron species index (more on Species in the next hands-on session) to the setup of the standard textbook object:

In [15]:
# Electrons are zeroth species by convention. They haven't been defined yet so here is where you put in the index.
rk.setStandardTextbookOptions(tempDerivSpeciesIDs=[0]) # This will enable us to use the temperature derivation for electrons directly

We'll use the built-in moment derivations to get the density and energy density of the electrons

Use the `densityMoment` and `energyMoment` derivations to calculate the n and W below. Both derivations require a single distribution variable passed to them

In [16]:
# n is Var and Dual but the derivation will automatically interpolate from cell centre f
# For derived variables you don't specify an initial value - it's calculated from the derivation based on some variables which already have
# an initial condition set. However, for post-processing this will mean the variable starts from zero as the first output step is done before
# any calculation. Because of this you may want to insert some kind of initial value so the plots look nicer.
rk.addVarAndDual("n",n, units='$10^{19} m^{-3}$',isDerived=True,derivationRule=sc.derivationRule("densityMoment",["f"])) # [YOUR CODE HERE]
rk.addVar("W",isDerived=True,derivationRule=sc.derivationRule("energyMoment",["f"])) # [YOUR CODE HERE] we only need the energy density at cell centers to get the temperature

The temperature derivation for the electrons will be generated with the name `tempFromEnergye`, and will require three variables passed to it, the energy density, particle density, as well as a particle flux. Since our problem setup has no flows, we can use a dummy variable instead of the flux.

In [17]:
# We have no flows and no flow variable but we have to tell this to the derivation. Here
# we create a a dummy variable that just contains a bunch of zeros.
# Units are used for post-processing only and not used in the code.
rk.addVar("zeroVar",isDerived=True,outputVar=False) # We can suppress the outputting of a variable using the outputVar flag
rk.addVarAndDual("T",units='$100eV$',isDerived=True,derivationRule=sc.derivationRule("tempFromEnergye",["W","n","zeroVar"]),isCommunicated=True) #[YOUR CODE HERE]

Finally, we add the electric field and time variables

In [18]:
rk.addVarAndDual("E",primaryOnDualGrid=True,isCommunicated=True)
rk.addVar("time",isScalar=True,isDerived=True)

### Adding the required models

The `addESModels` function can be used to add all the relevant terms and models for this example

In [19]:
# This just wraps a bunch of stuff to set up the Epperlein-Short problem.
addESModels(wrapper=rk,
            lMax=lMax,
            distFunName="f",
            eFieldName="E",
            elTempName="T",
            elDensName="n");

Checking terms in model adv:
   Checking term adv_plus1
   Checking term adv_minus2
Checking terms in model E-adv:
   Checking term eAdv_H1
   Checking term eAdv_G2
Checking terms in model AM:
   Checking term eTerm
Checking terms in model e-e0:
   Checking term dragTerm
   Checking term diffTerm
Checking terms in model e-i_odd:
   Checking term eiCollStationaryIons
Checking terms in model e-e_odd:
   Checking term 8pi*f0*fl
   Checking term diffTermI2
   Checking term diffTermJ2
   Checking term dfdv2
   Checking term termLL
   Checking term C1Il+2_h=2
   Checking term C1J-l-1_h=2
   Checking term C2Il_h=2
   Checking term C2J1-l_h=2
   Checking term C3Il+2_h=2
   Checking term C4J-l-1_h=2
   Checking term C5Il_h=2
   Checking term C6J1-l_h=2


In [20]:
addESModels?

Signature:
addESModels(
    wrapper: RMK_support.rk_wrapper.RKWrapper,
    lMax: int,
    distFunName: str,
    eFieldName: str,
    elTempName: str,
    elDensName: str,
) -> None
Docstring:
Adds the terms and models needed for the Epperlein-Short problem

Args:
    wrapper (RKWrapper): Wrapper to add the models to
    lMax (int): Higest present l harmonic
    distFunName (str): Name of the electron distribution function 
    eFieldName (str): Name of the electric field variable
    elTempName (str): Name of the electron temperature variable
    elDensName (str): Name of the electron density variable
File:      /home/ReMKiT1D-Workshop-2024/hands_on_sessions/es_models.py
Type:      function

### Adding heat flux diagnostics

We want to be able to inspect the deviation of the heat flux obtained from the electron distribution function with that predicted by the classical Braginskii model. 

Let's first add a variable for the heat flux using one of the pre-built moment derivations 

In [21]:
# Remember that fluxes live on  edges so you must set primaryOnDualGrid=True
rk.addVarAndDual("q",isDerived=True,primaryOnDualGrid=True,derivationRule=sc.derivationRule("heatFluxMoment",["f"]),isCommunicated=True) #[YOUR CODE HERE]

Let's get ReMKiT1D to calculate the Braginskii heat flux at each step. We will need the Coulomb logarithm, the conductivity, and the temperature gradient. Let's use a combination of built-in derivations and tree derivations to calculate this.

First, we need the Coulomb logarithm. An e-i Coulomb logarithm derivation is added for each ion species in a ReMKiT1D simulation. The `addESModels` function adds a species named `D+`, so we use the derivation name `logLeiD+`. It requires the electron temperature and density.

In [22]:
# We are putting this on cell centres cause we're using a centre grad later
rk.addVar("logLei",isDerived=True,derivationRule=sc.derivationRule("logLeiD+",["T","n"])) # [YOUR CODE HERE]

The conductivity $\kappa$ will be proportional to $T^{5/2}/\text{logL}$. For convenience, the constant of proportionality can be obtained using the `kappa0` function from `es_models.py`. We can use this to add a variable to calculate the heat Braginskii conductivity

In [23]:

rk.addVar("kappa",isDerived=True,derivationRule=sc.derivationRule("kappa",["T","logLei"]),derivOptions=treeDerivation(kappa0(rk)*Node("T")**(5/2)/Node("logLei"))) # [YOUR CODE HERE]

To construct the heatflux $q=-\kappa \nabla T$, we need the temperature gradient. One way of doing this is using the built-in gradient derivation `gradDeriv`, passing it the variable we want to calculate the gradient of, which should live on the regular grid.

In [24]:
# This one is on centres
rk.addVar("gradT",isDerived=True,derivationRule=sc.derivationRule("gradDeriv",["T"])) # [YOUR CODE HERE]

Finally we can assemble the heat flux using the two added variables `kappa` and `gradT` using the calculation tree approach.

In [25]:
rk.addVar("qT",isDerived=True,derivationRule=sc.derivationRule("qT",["kappa","gradT"]),derivOptions=treeDerivation(
    -Node("kappa")*Node("gradT"))) # [YOUR CODE HERE]

### Setting up the time integration options

Below is a standard setup for the Epperlein-Short test time integration with ReMKiT1D.

In [26]:
integrator = sc.picardBDEIntegrator(absTol=10.0, convergenceVars=["f"])

rk.addIntegrator("BE", integrator)

rk.setIntegratorGlobalData(initialTimestep=0.05)

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1", bdeStep.dict())

Nt = 300
rk.setFixedNumTimesteps(Nt)
rk.setFixedStepOutput(Nt/30)

rk.setPETScOptions(cliOpts="-pc_type bjacobi -sub_pc_factor_shift_type nonzero",kspSolverType="gmres")

### Create config 

In [24]:
rk.writeConfigFile()

In [ ]:
# NOTE: to debug, you can change the "build" directory to "debug" to access the debug build

### Data analysis

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

numFiles=30
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames, varsToIgnore=["zeroVar"]) # Ignore the variables that aren't in the output
loadedData


We can inspect fluid variables using the standard dashboard tool

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show()

A way to explore distribution variables is available using the `distDynMap` function

In [ ]:
dashboard.distDynMap().show()

Finally, we can compare the simulated and classical values of the heat flux

In [ ]:
dashboard.fluidMultiComparison(["q","qT"])